<a href="https://colab.research.google.com/github/WalnerPessoa/pytorch_challenge/blob/master/Walner_Image_Classifier_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing an AI application - WALNER DE OLIVIERA

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

--------
# Desenvolvendo um aplicativo do AI
Mais adiante, os algoritmos de IA serão incorporados em mais e mais aplicativos do dia a dia. Por exemplo, você pode querer incluir um classificador de imagem em um aplicativo de smartphone. Para fazer isso, você usaria um modelo de aprendizado profundo treinado em centenas de milhares de imagens como parte da arquitetura geral do aplicativo. Uma grande parte do desenvolvimento de software no futuro estará usando esses tipos de modelos como partes comuns das aplicações.

Neste projeto, você treinará um classificador de imagens para reconhecer diferentes espécies de flores. Você pode imaginar usando algo assim em um aplicativo de telefone que informa o nome da flor que sua câmera está olhando. Na prática, você treinaria esse classificador e o exportaria para uso em seu aplicativo. Nós estaremos usando este conjunto de dados de 102 categorias de flores, você pode ver alguns exemplos abaixo.

<img src='https://github.com/WalnerPessoa/pytorch_challenge/blob/master/assets/Flowers.png?raw=1' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

------
O projeto é dividido em várias etapas:

Carregar e pré-processar o conjunto de dados da imagem

Treine o classificador de imagens em seu conjunto de dados

Use o classificador treinado para prever o conteúdo da imagem

Nós vamos guiá-lo através de cada parte que você implementará em Python.

Quando você concluir este projeto, você terá um aplicativo que pode ser treinado em qualquer conjunto de imagens rotuladas. Aqui, sua rede aprenderá sobre flores e terminará como um aplicativo de linha de comando. Mas o que você faz com suas novas habilidades depende da sua imaginação e esforço na construção de um conjunto de dados. Por exemplo, imagine um aplicativo em que você tira uma foto de um carro, informa a marca e o modelo e procura informações sobre ele. Vá criar seu próprio conjunto de dados e faça algo novo.

A primeira é importar os pacotes que você precisa. É uma boa prática manter todas as importações no início do seu código. À medida que você trabalha nesse bloco de anotações e descobre que precisa importar um pacote, adicione a importação aqui.

In [0]:
# Ajuda de colega GabrielePicco   no projeto final
# https://github.com/GabrielePicco/deep-learning-flower-identifier
# Google Colaboratory / Online notebook
# https://drive.google.com/file/d/13_QyF9sghEIrc8iz4j5hPk0ssDE7-ExY/view

In [0]:
# script do ORIENTADOR @Julio Silva 

%%shell
# Another script offering for the God of Setup Boilerplates

rm -rf sample_data

# !wget -q -nc https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
# !unzip -q -o flower_data.zip
# !wget -q -nc https://raw.githubusercontent.com/joaopamaral/pytorch_challenge/master/cat_to_name.json



# Udacity train + valid dataset
wget -q -nc https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip -O flower_data.zip
unzip -q -o flower_data.zip
rm flower_data.zip

# Labeling asset
wget -q -nc https://raw.githubusercontent.com/udacity/pytorch_challenge/master/cat_to_name.json -O cat_to_name.json
# Cutout
wget -q -nc https://raw.githubusercontent.com/uoguelph-mlrg/Cutout/master/util/cutout.py -O cutout.py
# ImbalancedDatasetSampler: Oversampling & undersampling asset
wget -q -nc https://raw.githubusercontent.com/ufoym/imbalanced-dataset-sampler/master/sampler.py -O sampler.py

# Plotting helper
wget -q -nc https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py -O helper.py

# Test dataset from Kaggle @ http://www.robots.ox.ac.uk/~vgg/data/flowers/102/
wget -q -nc https://github.com/cahya-wirawan/Udacity-Course/blob/master/flower_test.tgz?raw=true -O flower_test.tgz
tar  -xvzf flower_test.tgz
# Try 'tar --help' or 'tar --usage' for more information.
mkdir flower_data/test
mv test/* flower_data/test
# Try 'mv --help' for more information.
rm flower_test.tgz
rm -rf test

pip install torchvision
pip install matplotlib==2.0.2 Pillow==5.3.0

In [1]:
!wget -q -nc https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -q -o flower_data.zip
!wget -q -nc https://raw.githubusercontent.com/joaopamaral/pytorch_challenge/master/cat_to_name.json

#Install Torch in Collab

# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
!wget https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py

tcmalloc: large alloc 1073750016 bytes == 0x5797a000 @  0x7f3d833952a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
--2019-01-07 05:06:08--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2813 (2.7K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   2.75K  --.-KB/s    in 0s      

2019-01-07 05:06:08 (58.5 MB/s) - ‘helper.py’ saved [2813/2813]



In [0]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


from PIL import Image
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import torch
from torch import nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import models, datasets
import torchvision.transforms as transforms
#from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

import helper

import PIL

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [2]:
!ls /content/flower_data
print(os.listdir("/content/flower_data"))

train  valid
['train', 'valid']


In [3]:
print(PIL.PILLOW_VERSION)
# Above should print 5.3.0 , if it still prints 4.0 then just restart your runtime from the Menu > Runtime > Restart Runtime. 
# Once that is done, Google Colab will load the latest one and this error won’t happen again.


5.4.1


In [0]:
# https://medium.com/@ml_kid/dealing-with-pil-image-has-no-attribute-register-extensions-7d79b4e7dcf0

#### we need pillow version of 5.3.0
#### we will uninstall the older version first
#!pip uninstall -y Pillow
#### install the new one
#!pip install Pillow==5.3.0
#### import the new one
#import PIL
#print(PIL.PILLOW_VERSION)
#### this should print 5.3.0. If it doesn't, then restart your runtime:
#### Menu > Runtime > Restart Runtime

In [4]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    

CUDA is available!  Training on GPU ...


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.


------
Aqui você usará `torchvision` para carregar os dados da Documentação. Você pode baixar os dados aqui.

O conjunto de dados é dividido em duas partes, treinamento (train)e validação(valid). Para o treinamento, você desejará aplicar transformações como dimensionamento, recorte e inversão aleatórios. Isso ajudará a generalizar a rede levando a um melhor desempenho. Se você usar uma rede pré-treinada, também precisará garantir que os dados de entrada sejam redimensionados para 224x224 pixels, conforme exigido pelas redes.

O conjunto de validação é usado para medir o desempenho do modelo em dados que ainda não foram vistos. Para isso, você não deseja nenhuma transformação de escala ou rotação, mas precisará redimensioná-las e recortar as imagens no tamanho apropriado.

As redes pré-treinadas disponíveis no `torchvision` foram treinadas no banco de dados ImageNet, onde cada canal de cores foi normalizado separadamente. Para ambos os conjuntos, você precisará normalizar as médias e os desvios padrão das imagens para o que a rede espera. Para os meios, é `[0.485, 0.456, 0.406]` e para os desvios padrão `[0.229, 0.224, 0.225]`, calculados a partir das imagens do ImageNet. Esses valores irão deslocar cada canal de cor para ser centralizado em 0 e variar de -1 a 1.

In [0]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
dirs = {'train': train_dir, 
        'valid': valid_dir}

In [0]:
# APAGAR #################

# load and transform data using ImageFolder

# VGG-16 Takes 224x224 images as input, so we resize all of them
data_transform = data_transforms = {
    'train':transforms.Compose([transforms.RandomResizedCrop(224), 
                                      transforms.ToTensor()]),
    'valid':transforms.Compose([transforms.RandomResizedCrop(224), 
                                      transforms.ToTensor()
    ])}

image_datasets= {'train': datasets.ImageFolder(dirs['train'], transform=data_transform),
                 'valid': datasets.ImageFolder(dirs['valid'], transform=data_transform)}
batch_size = 64
num_workers=8
dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True),
               'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)}

dataset_sizes = {'train': len(image_datasets['train']),
                 'valid': len(image_datasets['valid'])
                }
class_names = image_datasets['train'].classes 

In [0]:
# walner

################################################################
# TODO: Define your transforms for the training and validation sets
# data_transforms 

data_transforms = data_transforms = {
    'train': transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))
    ]),
    'valid': transforms.Compose([
    transforms.Resize(255 + 23),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))
    ])}

################################################################
# TODO: Load the datasets with ImageFolder
# image_datasets   
  
image_datasets = {'train':datasets.ImageFolder(dirs['train'], transform=data_transforms['train']),
                  'valid':datasets.ImageFolder(dirs['valid'], transform=data_transforms['valid'])
                 }

################################################################
# TODO: Load the datasets with ImageFolder
# image_datasets 
# prepare data loaders

# define dataloader parameters

batch_size = 32
num_workers=0
# -----------------

dataloaders = {'train':torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, num_workers=num_workers, shuffle=True),
              'valid':torch.utils.data.DataLoader(image_datasets['valid'], batch_size=batch_size, num_workers=num_workers, shuffle=True)
             }


################################################################              
dataset_sizes = {'train': len(image_datasets['train']),
                 'valid': len(image_datasets['valid'])
                }
################################################################
class_names = image_datasets['train'].classes                     
                 
####    other form to do using for option an variable 'x'
####
####    image_datasets = {x: datasets.ImageFolder(dirs[x],   transform=data_transforms[x]) for x in ['train', 'valid']}
####    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'valid']}
####    dataset_sizes = {x: len(image_datasets[x])  for x in ['train', 'valid']}
####    class_names = image_datasets['train'].classes 

In [8]:
print('Num classe names: ',len(class_names))
print('Num training images: ', len(image_datasets['train']))
print('Num valid images: ', len(image_datasets['valid']))
print(dataset_sizes)

Num classe names:  102
Num training images:  6552
Num valid images:  818
{'train': 6552, 'valid': 818}


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

---------

Mapeamento de etiquetas

Você também precisará carregar um mapeamento do rótulo da categoria para o nome da categoria. Você pode encontrar isto no arquivo `cat_to_name.json`. É um objeto JSON que você pode ler com o módulo [`json`] (https://docs.python.org/2/library/json.html). Isso lhe dará um dicionário mapeando as categorias codificadas por números inteiros para os nomes reais das flores.

In [0]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [10]:
print(cat_to_name)

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

In [11]:
print(dataloaders['train'])

In [12]:
# Visualize some sample data

# obtain one batch of training images
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    
    # doing the reverse operation to remove normalization

    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = np.transpose(images[idx], (1, 2, 0))
    image = image*std+mean
    
    plt.imshow(image)
    
    #plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    #ax.set_title(class_names[labels[idx]])
    ax.set_title(cat_to_name[str(labels[idx].item())])
    

ValueError: ignored

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

---------
# Construindo e treinando o classificador

Agora que os dados estão prontos, é hora de construir e treinar o classificador. Como de costume, você deve usar um dos modelos pré-concebidos da torchvision.models para obter os recursos da imagem. Crie e treine um novo classificador de feed-forward usando esses recursos.

Nós vamos deixar essa parte para você. Se você quiser conversar com alguém, converse com seus colegas! Você também pode fazer perguntas nos fóruns ou se juntar aos instrutores em horário de expediente.

Consulte The Rubic para orientação sobre como completar com sucesso esta seção. 

Coisas que você precisa fazer:
*Carregue uma rede pré-treinada. Carregue uma rede pré-treinada. 
(Se você precisa de um ponto de partida, as redes VGG funcionam muito bem e são fáceis de usar)
* Definir uma nova rede de feed-forward não treinada como um classificador, usando ativações de ReLU e dropout
* Treinar as camadas do classificador usando backpropagation usando a rede pré-treinada para obter os recursos
* Acompanhe a perda e precisão no conjunto de validação para determinar os melhores hiperparâmetros

Nós deixamos uma célula aberta para você abaixo, mas use quantos você precisar. Nosso conselho é dividir o problema em partes menores que você pode executar separadamente. Verifique se cada parte está fazendo o que você espera e, em seguida, passe para a próxima. Você provavelmente descobrirá que, ao trabalhar em cada parte, precisará voltar e modificar seu código anterior. Isso é totalmente normal!

Ao treinar, certifique-se de atualizar apenas os pesos da rede de feed-forward. Você deve ser capaz de obter a precisão de validação acima de 70% se você construir tudo certo. Certifique-se de tentar diferentes hyperparameters (taxa de aprendizagem, unidades no classificador, épocas, etc) para encontrar o melhor modelo. Salve esses hiperparâmetros para usar como valores padrão na próxima parte do projeto.




## [TORCHVISION.MODELS](https://pytorch.org/docs/stable/torchvision/models.html)

The models subpackage contains definitions for the following model architectures:

O subpackage de modelos contém definições para as seguintes arquiteturas de modelo:

* AlexNet
* VGG
* ResNet
* SqueezeNet
* DenseNet
* Inception v3



In [0]:
# TODO: Build and train your network
# https://colab.research.google.com/drive/1_vVJUhHL_8fnkpfqfNRQzppQQs89LWyc#scrollTo=iIHxs7pWVmMs


## Define the Model


In [13]:
# Load the pretrained model from pytorch
model_walner = models.vgg16(pretrained=True)

# print out the model structure
print(model_walner)

# Freeze training for all "features" layers
for param in model_walner.features.parameters():
    param.requires_grad = False
    
# freeze all pretrained model parameters 
#for param in model.parameters():
#    param.requires_grad_(False)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:11<00:00, 47233087.45it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [14]:
print(model_walner.classifier[6].in_features) 
print(model_walner.classifier[6].out_features) 

4096
1000


### Final Classifier Layer


In [0]:
# MODEL OPTION 1 
n_inputs = model_walner.classifier[6].in_features
last_layer = nn.Linear(n_inputs, len(class_names))
model_walner.classifier[6] = last_layer
if train_on_gpu:
    model_walner.cuda()
print(model_walner.classifier[6].out_features)

102


In [0]:
# MODEL OPTION 2
classifier = nn.Sequential(OrderedDict([
                          ('0', nn.Linear(25088, 4096)),
                          ('1', nn.ReLU()),
                          ('2', nn.Dropout(0.5)),
                          ('3', nn.Linear(4096, 4096)),
                          ('4', nn.ReLU()),
                          ('5', nn.Dropout(0.5)),
                          ('6', nn.Linear(4096, 102)),
                          ('7', nn.LogSoftmax(dim=1))
                          ]))
model_walner.classifier = classifier

In [16]:
# if GPU is available, move the model to GPU
if train_on_gpu:
    model_walner.cuda()
# check to see that your last layer produces the expected number of outputs
print(model_walner.classifier[6].out_features)
#print(model_walner)

102


In [17]:
print(model_walner)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

### Specify Loss Function and Optimizer

In [0]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
#criterion = nn.CrossEntropyLoss()

# Criteria NLLLoss which is recommended with Softmax final layer
criterion = nn.NLLLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
#optimizer = optim.SGD(model_walner.classifier.parameters(), lr=0.001)

# Observe that all parameters are being optimized
optimizer = optim.Adam(model_walner.classifier.parameters(), lr=0.001)

## Training


In [30]:
# number of epochs to train the model
# Número de épocas para treinar o modelo

n_epochs = 2

valid_loss_min = np.Inf # track change in validation loss ------- new


for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    # acompanhar a perda de treinamento e validação

    train_loss = 0.0
    
    ###################
    # train the model #  
    # treinar o modelo #
    ###################
    # model by default is set to train
    # model por padrão está definido para treinar
    for batch_i, (data, target) in enumerate(dataloaders['train']):
        # move tensors to GPU if CUDA is available
        # move tensores para GPU se CUDA estiver disponível
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            
        # clear the gradients of all optimized variables
        # limpar os gradientes de todas as variáveis otimizadas
        optimizer.zero_grad()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        # forward pass: calcula as saídas previstas passando as entradas ao modelo
        output = model_walner(data)
        
        # calculate the batch loss
        # calcular a perda do lote
        loss = criterion(output, target)
        
        # backward pass: compute gradient of the loss with respect to model parameters
        # backward pass: calcula o gradiente da perda em relação aos parâmetros do modelo
        loss.backward()
        
        # perform a single optimization step (parameter update)
        # executar uma única etapa de otimização (atualização de parâmetro)
        optimizer.step()
        
        # update training loss 
        # atualização da perda de treinamento
        train_loss += loss.item()
        
        if batch_i % 20 == 19:    
        # print training loss every specified number of mini-batches
        # imprime a perda de treinamento a cada número especificado de mini-lotes
            print('Epoch %d, Batch %d Training Loss: %.16f' %
                  (epoch, batch_i + 1, train_loss / 20))
            train_loss = 0.0

Epoch 1, Batch 20 Training Loss: 0.2597504957579077
Epoch 1, Batch 40 Training Loss: 0.2049192842096090
Epoch 1, Batch 60 Training Loss: 0.1966737287119031
Epoch 1, Batch 80 Training Loss: 0.2501639008522034
Epoch 1, Batch 100 Training Loss: 0.2764779623597861
Epoch 1, Batch 120 Training Loss: 0.3016494004055857
Epoch 1, Batch 140 Training Loss: 0.2665647838264704
Epoch 1, Batch 160 Training Loss: 0.3237644335255027
Epoch 1, Batch 180 Training Loss: 0.2578768536448479
Epoch 1, Batch 200 Training Loss: 0.2724270279519260
Epoch 2, Batch 20 Training Loss: 0.2212273521348834
Epoch 2, Batch 40 Training Loss: 0.1882643240038306
Epoch 2, Batch 60 Training Loss: 0.1893814264563844
Epoch 2, Batch 80 Training Loss: 0.1995665056165308
Epoch 2, Batch 100 Training Loss: 0.2078848308883607
Epoch 2, Batch 120 Training Loss: 0.2806867137551308
Epoch 2, Batch 140 Training Loss: 0.2415180532261729
Epoch 2, Batch 160 Training Loss: 0.2851555634289980
Epoch 2, Batch 180 Training Loss: 0.2736269693821669
E

## Validated
https://github.com/udacity/deep-learning-v2-pytorch/blob/master/transfer-learning/Transfer_Learning_Solution.ipynb

In [31]:
###### OPÇÃO 1 => CODE VALIDATE 



# over 102 flower classes
# mais de 102 classes de flores

valid_loss = 0.0           
class_correct = list(0. for i in range(102))
class_total = list(0. for i in range(102))

model_walner.eval() # eval mode / Modo eval

print('batch: ',batch_size)
print('dataset: ',len(dataloaders['valid'].dataset))
#
#
######################
# Validate the model #  
# validar o modelo   #
######################
#
# model by default is set to Validate
# iterate over valid data (validação)
# iterar sobre dados de validação



##### for data, target in dataloaders['valid']:
for batch_i, (data, target) in enumerate(dataloaders['valid']):


    # move tensors to GPU if CUDA is available
    # move tensores para GPU se CUDA estiver disponível
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()

    # forward pass: compute predicted outputs by passing inputs to the model
    # forward pass: calcula as saídas previstas 
    output = model_walner(data)

    # calculate the batch loss
    # calcula a perda do lote
    loss = criterion(output, target)

    # update  valid loss
    # atualização de perda de validação
    valid_loss += loss.item()*data.size(0)

    


    # convert output probabilities to predicted class
    # converter probabilidades de saída em classes previstas
    _, pred = torch.max(output, 1)    

    # compare predictions to true label
    # compare previsões para o rótulo verdadeiro
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())

    # calculate valid accuracy for each object class
    # calcular a precisão do validação para cada classe de objeto
    
    #print('numeração: ',batch_i)#------------------- new comment
    #print('correct: ',correct)#------------------- new comment

    #for i in range(batch_size):
    for i in range(len(target.data)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate avg valid loss'
# calcular a perda do validação médio
valid_loss = valid_loss/len(dataloaders['valid'].dataset)
print('Average Valid Loss: {:.6f}\n'.format(valid_loss))

for i in range(102):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            class_names[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no validation examples)' % (class_names[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))


#ax.set_title(class_names[labels[idx]])
#ax.set_title(cat_to_name[str(labels[idx].item())])
  

batch:  32
dataset:  818
Average Valid Loss: 0.953228

Test Accuracy of     1: 75% ( 6/ 8)
Test Accuracy of    10: 100% ( 4/ 4)
Test Accuracy of   100: 100% ( 6/ 6)
Test Accuracy of   101: 100% ( 5/ 5)
Test Accuracy of   102: 100% ( 6/ 6)
Test Accuracy of    11: 70% ( 7/10)
Test Accuracy of    12: 80% ( 4/ 5)
Test Accuracy of    13: 60% ( 3/ 5)
Test Accuracy of    14: 100% ( 1/ 1)
Test Accuracy of    15: 100% ( 7/ 7)
Test Accuracy of    16: 50% ( 1/ 2)
Test Accuracy of    17: 87% (14/16)
Test Accuracy of    18: 63% ( 7/11)
Test Accuracy of    19: 75% ( 3/ 4)
Test Accuracy of     2: 100% ( 6/ 6)
Test Accuracy of    20: 57% ( 4/ 7)
Test Accuracy of    21: 75% ( 3/ 4)
Test Accuracy of    22: 100% ( 8/ 8)
Test Accuracy of    23: 100% (12/12)
Test Accuracy of    24: 100% ( 5/ 5)
Test Accuracy of    25: 100% ( 2/ 2)
Test Accuracy of    26: 66% ( 2/ 3)
Test Accuracy of    27: 100% ( 1/ 1)
Test Accuracy of    28: 60% ( 3/ 5)
Test Accuracy of    29: 85% ( 6/ 7)
Test Accuracy of     3: 50% ( 1/ 

In [0]:
###### OPÇÃO 2 => CODE TRAINING & VALIDATE 


# number of epochs to train the model
# Número de épocas para treinar o modelo

n_epochs = 2

valid_loss_min = np.Inf # track change in validation loss ------- new


for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    # acompanhar a perda de treinamento e validação

    train_loss = 0.0
    valid_loss = 0.0 # ------------------------------------------new
    
    ###################
    # train the model #  
    # treinar o modelo #
    ###################
    # model by default is set to train
    # model por padrão está definido para treinar
    
    model_walner.train()  # -------------------------------------new
    for batch_i, (data, target) in enumerate(dataloaders['train']):
        # move tensors to GPU if CUDA is available
        # move tensores para GPU se CUDA estiver disponível
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            
        # clear the gradients of all optimized variables
        # limpar os gradientes de todas as variáveis otimizadas
        optimizer.zero_grad()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        # forward pass: calcula as saídas previstas passando as entradas ao modelo
        output = model_walner(data)
        
        # calculate the batch loss
        # calcular a perda do lote
        loss = criterion(output, target)
        
        # backward pass: compute gradient of the loss with respect to model parameters
        # backward pass: calcula o gradiente da perda em relação aos parâmetros do modelo
        loss.backward()
        
        # perform a single optimization step (parameter update)
        # executar uma única etapa de otimização (atualização de parâmetro)
        optimizer.step()
        
        # update training loss 
        # atualização da perda de treinamento
        train_loss += loss.item()
        # train_loss += loss.item()*data.size(0)# -------new comment
        
        #if batch_i % 20 == 19:    
        # print training loss every specified number of mini-batches
        # imprime a perda de treinamento a cada número especificado de mini-lotes
        #    print('Epoch %d, Batch %d loss: %.16f' %
        #          (epoch, batch_i + 1, train_loss / 20))
        #    train_loss = 0.0








    # over 102 flower classes
    # mais de 102 classes de flores

    # valid_loss = 0.0           ------------------- new comment
    class_correct = list(0. for i in range(102))
    class_total = list(0. for i in range(102))

    model_walner.eval() # eval mode / Modo eval
    
    print('batch: ',batch_size)
    print('dataset: ',len(dataloaders['valid'].dataset))
    #
    #
    ######################
    # Validate the model #  
    # validar o modelo   #
    ######################
    #
    # model by default is set to Validate
    # iterate over valid data (validação)
    # iterar sobre dados de validação



    ##### for data, target in dataloaders['valid']:
    for batch_i, (data, target) in enumerate(dataloaders['valid']):


        # move tensors to GPU if CUDA is available
        # move tensores para GPU se CUDA estiver disponível
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        # forward pass: compute predicted outputs by passing inputs to the model
        # forward pass: calcula as saídas previstas 
        output = model_walner(data)

        # calculate the batch loss
        # calcula a perda do lote
        loss = criterion(output, target)

        # update  valid loss
        # atualização de perda de validação
        valid_loss += loss.item()*data.size(0)

        # ---------------------------------------------------------------new
        # calculate average losses # new
        train_loss = train_loss/len(dataloaders['train'].dataset) # new
        valid_loss = valid_loss/len(dataloaders['valid'].dataset) # new

        # print training/validation statistics  # new
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss)) # new

        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model_walner.state_dict(), 'model_walner_augmented.pt')
            valid_loss_min = valid_loss

        # ---------------------------------------------------------------new


        # convert output probabilities to predicted class
        # converter probabilidades de saída em classes previstas
        _, pred = torch.max(output, 1)    

        # compare predictions to true label
        # compare previsões para o rótulo verdadeiro
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())

        # calculate valid accuracy for each object class
        # calcular a precisão do validação para cada classe de objeto
        #print('numeração: ',batch_i)#------------------- new comment
        #print('correct: ',correct)#------------------- new comment

        #for i in range(batch_size):
        for i in range(len(target.data)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
        '''
        # calculate avg valid loss'
        # calcular a perda do validação médio
        valid_loss = valid_loss/len(dataloaders['valid'].dataset)
        print('Test Loss: {:.6f}\n'.format(valid_loss))

        for i in range(102):
            if class_total[i] > 0:
                print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                    class_names[i], 100 * class_correct[i] / class_total[i],
                    np.sum(class_correct[i]), np.sum(class_total[i])))
            else:
                print('Test Accuracy of %5s: N/A (no validation examples)' % (class_names[i]))

        print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
            100. * np.sum(class_correct) / np.sum(class_total),
            np.sum(class_correct), np.sum(class_total)))
        '''

In [0]:
#
# sugestão do JAder
# torch.load(ckpt_file, map_location=lambda storage, loc: storage)
#

### Visualize Sample Test Results

In [27]:
# obtain one batch of test images
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if train_on_gpu:
    images = images.cuda()

# get sample outputs
output = model_walner(images)
# convert output probabilities to predicted class
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title("{} ({})".format(class_names[preds[idx]], class_names[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))

ValueError: ignored

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

--------
# Salve o ponto de verificação

Agora que sua rede está treinada, salve o modelo para que você possa carregá-lo posteriormente para fazer previsões. Você provavelmente quer salvar outras coisas, como o mapeamento de classes para índices que você obtém de um dos conjuntos de dados da imagem: `image_datasets ['train']. Class_to_idx`. Você pode anexar isso ao modelo como um atributo que facilita a inferência mais tarde.

```model.class_to_idx = image_datasets['train'].class_to_idx```


Lembre-se de que você vai querer reconstruir completamente o modelo mais tarde para poder usá-lo para inferência. Certifique-se de incluir qualquer informação que você precisa no checkpoint. Se você quiser carregar o modelo e continuar treinando, será necessário salvar o número de épocas, bem como o estado do otimizador, `optimizer.state_dict`. Você provavelmente vai querer usar esse modelo treinado na próxima parte do projeto, então é melhor salvá-lo agora.

In [0]:
# TODO: Save the checkpoint 
model_file_name = 'classifier_walner.pt'
model_walner.class_to_idx = image_datasets['train'].class_to_idx
model_walner.cuda()
torch.save(model_walner.state_dict(), 'classifier_walner.pt')

#torch.save({'arch': 'vgg16',
#            'state_dict': model.state_dict(), 
#            'class_to_idx': model_walner.class_to_idx}, 
#             model_file_name)
#torch.save({'arch': 'vgg19',
#            'state_dict': model.state_dict(), 
#            'class_to_idx': model.class_to_idx}, 
#             model_file_name)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

-------
## Carregando o ponto de verificação

Neste ponto, é bom escrever uma função que possa carregar um ponto de verificação e reconstruir o modelo. Dessa forma, você pode voltar a este projeto e continuar trabalhando nele sem precisar reciclar a rede.

In [0]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
model_walner.load_state_dict(torch.load('classifier_walner.pt'))
# archive salving during validated
# model_walner.load_state_dict(torch.load('model_walner_augmented.pt'))


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

_____

# Inferência para classificação

Agora você vai escrever uma função para usar uma rede treinada para inferência. Ou seja, você vai passar uma imagem para a rede e prever a classe da flor na imagem. Escreva uma função chamada `predict` que leva uma imagem e um modelo, então retorna as classes mais prováveis de $ K $ juntamente com as probabilidades. Deve parecer

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

-------
Primeiro, você precisará manipular o processamento da imagem de entrada de forma que ela possa ser usada em sua rede.

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.


------
## Preprocessamento de Imagem

Você vai querer usar `PIL` para carregar a imagem [documentação]. É melhor escrever uma função que pré-processe a imagem para que ela possa ser usada como entrada para o modelo. Esta função deve processar as imagens da mesma maneira usada. para treinamento.

Primeiro, redimensione as imagens em que o lado mais curto é de 256 pixels, mantendo a proporção. Isso pode ser feito com os métodos [`thumbnail`] ou [` redimensionar`]. Em seguida, você precisará recortar a parte central 224x224 da imagem.

Canais de cores de imagens são tipicamente codificados como inteiros 0-255, mas o modelo esperado flutua 0-1. Você precisará converter os valores. É mais fácil com uma matriz Numpy, que você pode obter de uma imagem PIL, como `np_image = np.array (pil_image)`.

Como antes, a rede espera que as imagens sejam normalizadas de uma maneira específica. Para os meios, é `[0.485, 0.456, 0.406]` e para os desvios padrão `[0.229, 0.224, 0.225]`. Você vai querer subtrair os meios de cada canal de cor e dividir pelo desvio padrão.

E finalmente, o PyTorch espera que o canal de cores seja a primeira dimensão, mas é a terceira dimensão na imagem PIL e na matriz Numpy. Você pode reordenar as dimensões usando [`ndarray.transpose`] O canal de cor precisa ser o primeiro e manter a ordem das outras duas dimensões.

## PARTE INCREMENTAL DE WALNER

### Test the Trained Network

Test your trained model on previously unseen data! A "good" result will be a CNN that gets around 70% (or more, try your best!) accuracy on these test images.

In [0]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for batch_idx, (data, target) in enumerate(test_loader):
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

In [0]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

-----
Para verificar seu trabalho, a função abaixo converte um tensor de PyTorch e o exibe no notebook. Se a função `process_image` funcionar, a execução da saída por meio dessa função deve retornar a imagem original (exceto as porções recortadas).

In [0]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.


-----
## Class Prediction

Depois de conseguir imagens no formato correto, é hora de escrever uma função para fazer previsões com o seu modelo. Uma prática comum é prever as 5 ou mais classes mais prováveis (geralmente chamadas de top-$ K $). Você vai querer calcular as probabilidades da classe, em seguida, encontrar os maiores valores de $ K $.

Para obter os maiores valores de $ K $ em um tensor use [`x.topk (k)`]. Este método retorna as probabilidades mais altas de `k` e os índices dessas probabilidades correspondentes às classes. Você precisa converter esses índices para os rótulos de classe reais usando `class_to_idx`, que esperamos que você tenha adicionado ao modelo ou de um` ImageFolder` que você usou para carregar os dados ([veja aqui]. Certifique-se de inverter o dicionário para obter um mapeamento de índice para classe também.

Novamente, esse método deve pegar um caminho para uma imagem e um ponto de verificação de modelo e, em seguida, retornar as probabilidades e classes.


```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [0]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='https://github.com/WalnerPessoa/pytorch_challenge/blob/master/assets/inference_example.png?raw=1' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

------

## Verificação de sanidade

Agora que você pode usar um modelo treinado para previsões, verifique se isso faz sentido. Mesmo que a precisão da validação seja alta, é sempre bom verificar se não há erros óbvios. Use o `matplotlib` para plotar as probabilidades das 5 principais classes como um gráfico de barras, junto com a imagem de entrada. Deve ficar assim:

<IMAGEM>
  
Você pode converter da codificação de inteiro de classe para nomes de flores reais com o arquivo `cat_to_name.json` (deveria ter sido carregado anteriormente no bloco de notas). Para mostrar um tensor PyTorch como uma imagem, use a função `imshow` definida acima.

In [0]:
# TODO: Display an image along with the top 5 classes